# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from tqdm import tqdm
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import RFE, SelectKBest, mutual_info_regression, SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import shap
from IPython.display import display, HTML
import random
import operator
import numpy as np
import matplotlib.pyplot as plt
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
pd.set_option("display.max_rows", 50, "display.max_columns", 50)


KeyboardInterrupt: 

## Stats Util Functions

In [ ]:

def normalized_rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    range_y = np.max(y_true) - np.min(y_true)
    return rmse / (range_y + 1e-8)


def normalized_mae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    range_y = np.max(y_true) - np.min(y_true)
    return mae / (range_y + 1e-8)

## Decide to use additional google search trends
### additional google search trends are given as a sum per week, so remaining calculations and model will need to follow as such

In [ ]:
ADDITIONAL = True

# Dataframe Setup

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v3/location/US_GA.csv')
date = 'date'
df[date] = pd.to_datetime(df[date], format='%Y-%m-%d')

In [ ]:
df2 = pd.read_csv('/kaggle/input/additional-google-search/google1.csv')
df3 = pd.read_csv('/kaggle/input/additional-google-search/google2.csv')
df4 = pd.read_csv('/kaggle/input/additional-google-search/google3.csv')
df_search = pd.merge(pd.merge(df2, df3, on='Week', how='inner'), df4, on='Week', how='inner')

def convert_to_numeric(value):
    return pd.to_numeric(value.replace('<1', '0'))

df_search.iloc[:, 1:] = df_search.iloc[:, 1:].applymap(convert_to_numeric)
columns_to_convert =  [
    'Vaccine: (Georgia)',
    'COVID-19 vaccine: (Georgia)',
    'covid: (Georgia)',
    'covid vaccine: (Georgia)',
    'covid 19: (Georgia)',
    'georgia covid vaccine: (Georgia)',
    'georgia covid: (Georgia)_x',
    'vaccine near me: (Georgia)',
    'covid 19 georgia: (Georgia)',
    'ga covid 19: (Georgia)',
    'vaccine covid: (Georgia)',
    'covid ga: (Georgia)',
    'georgia covid: (Georgia)_y',
    'CVS vaccine: (Georgia)',
    'Coronavirus disease 2019: (Georgia)'
]
df_search[columns_to_convert] = df_search[columns_to_convert].apply(pd.to_numeric, errors='coerce')
df_search[date] = pd.to_datetime(df_search['Week'], format='%Y-%m-%d')
df_search.drop(columns=['Week'], inplace=True)

column_mapping = {
    'Vaccine: (Georgia)': 'search_additional_Vaccine',
    'COVID-19 vaccine: (Georgia)': 'search_additional_COVID-19_vaccine',
    'covid: (Georgia)': 'search_additional_covid',
    'covid vaccine: (Georgia)': 'search_additional_covid_vaccine',
    'covid 19: (Georgia)': 'search_additional_covid_19',
    'georgia covid vaccine: (Georgia)': 'search_additional_georgia_covid_vaccine',
    'georgia covid: (Georgia)_x': 'search_additional_georgia_covid_x',
    'vaccine near me: (Georgia)': 'search_additional_vaccine_near_me',
    'covid 19 georgia: (Georgia)': 'search_additional_covid_19_georgia',
    'ga covid 19: (Georgia)': 'search_additional_ga_covid_19',
    'vaccine covid: (Georgia)': 'search_additional_vaccine_covid',
    'covid ga: (Georgia)': 'search_additional_covid_ga',
    'georgia covid: (Georgia)_y': 'search_additional_georgia_covid_y',
    'CVS vaccine: (Georgia)': 'search_additional_CVS_vaccine',
    'Coronavirus disease 2019: (Georgia)': 'search_additional_Coronavirus_disease_2019',
}

df_search.rename(columns=column_mapping, inplace=True)

### Can choose between 2 label options

In [ ]:
label = 'new_vaccine_doses_administered'
# label = 'cumulative_vaccine_doses_administered'

## Imputation Strategy for Nulls

In [ ]:
df.isna().sum()

In [ ]:
if ADDITIONAL:
    print(df_search.isna().sum())

In [ ]:
cdh = [
    "new_confirmed",
    "new_deceased",
    "new_tested",
    "cumulative_confirmed",
    "cumulative_deceased",
    "cumulative_tested",
    "new_hospitalized_patients",
    "cumulative_hospitalized_patients",
    "current_hospitalized_patients",
    "new_intensive_care_patients",
    "cumulative_intensive_care_patients",
    "current_intensive_care_patients"
]

cdh_age = [
    "new_confirmed_age_0",
    "new_confirmed_age_1",
    "new_confirmed_age_2",
    "new_confirmed_age_3",
    "new_confirmed_age_4",
    "new_confirmed_age_5",
    "new_confirmed_age_6",
    "new_confirmed_age_7",
    "new_confirmed_age_8",
    "new_confirmed_age_9",
    "cumulative_confirmed_age_0",
    "cumulative_confirmed_age_1",
    "cumulative_confirmed_age_2",
    "cumulative_confirmed_age_3",
    "cumulative_confirmed_age_4",
    "cumulative_confirmed_age_5",
    "cumulative_confirmed_age_6",
    "cumulative_confirmed_age_7",
    "cumulative_confirmed_age_8",
    "cumulative_confirmed_age_9",
    "new_deceased_age_0",
    "new_deceased_age_1",
    "new_deceased_age_2",
    "new_deceased_age_3",
    "new_deceased_age_4",
    "new_deceased_age_5",
    "new_deceased_age_6",
    "new_deceased_age_7",
    "new_deceased_age_8",
    "new_deceased_age_9",
    "cumulative_deceased_age_0",
    "cumulative_deceased_age_1",
    "cumulative_deceased_age_2",
    "cumulative_deceased_age_3",
    "cumulative_deceased_age_4",
    "cumulative_deceased_age_5",
    "cumulative_deceased_age_6",
    "cumulative_deceased_age_7",
    "cumulative_deceased_age_8",
    "cumulative_deceased_age_9",
    "new_hospitalized_patients_age_0",
    "new_hospitalized_patients_age_1",
    "new_hospitalized_patients_age_2",
    "new_hospitalized_patients_age_3",
    "new_hospitalized_patients_age_4",
    "new_hospitalized_patients_age_5",
    "new_hospitalized_patients_age_6",
    "new_hospitalized_patients_age_7",
    "new_hospitalized_patients_age_8",
    "new_hospitalized_patients_age_9",
    "cumulative_hospitalized_patients_age_0",
    "cumulative_hospitalized_patients_age_1",
    "cumulative_hospitalized_patients_age_2",
    "cumulative_hospitalized_patients_age_3",
    "cumulative_hospitalized_patients_age_4",
    "cumulative_hospitalized_patients_age_5",
    "cumulative_hospitalized_patients_age_6",
    "cumulative_hospitalized_patients_age_7",
    "cumulative_hospitalized_patients_age_8",
    "cumulative_hospitalized_patients_age_9"
]
pop = [
    'population',
    'population_male',
    'population_female',
    'population_age_00_09',
    'population_age_10_19',
    'population_age_20_29',
    'population_age_30_39',
    'population_age_40_49',
    'population_age_50_59',
    'population_age_60_69',
    'population_age_70_79',
    'population_age_80_and_older'
]

mob = [
    'mobility_retail_and_recreation',
    'mobility_grocery_and_pharmacy',
    'mobility_parks',
    'mobility_transit_stations',
    'mobility_workplaces',
    'mobility_residential'
]

misc = [
    'school_closing',
    'workplace_closing',
    'cancel_public_events',
    'restrictions_on_gatherings',
    'public_transport_closing',
    'stay_at_home_requirements',
    'restrictions_on_internal_movement',
    'international_travel_controls',
    'income_support',
    'debt_relief',
    'fiscal_measures',
    'international_support',
    'public_information_campaigns',
    'testing_policy',
    'contact_tracing',
    'emergency_investment_in_healthcare',
    'investment_in_vaccines',
    'facial_coverings',
    'vaccination_policy',
    'stringency_index'
]
search = [col for col in df.columns if col.startswith("search_trends")]
weather = [
    'average_temperature_celsius',
    'minimum_temperature_celsius',
    'maximum_temperature_celsius',
    'rainfall_mm',
    'dew_point',
    'relative_humidity'
]

search_additional = [
    'search_additional_Vaccine',
    'search_additional_COVID-19_vaccine',
    'search_additional_covid',
    'search_additional_covid_vaccine',
    'search_additional_covid_19',
    'search_additional_georgia_covid_vaccine',
    'search_additional_georgia_covid_x',
    'search_additional_vaccine_near_me',
    'search_additional_covid_19_georgia',
    'search_additional_ga_covid_19',
    'search_additional_vaccine_covid',
    'search_additional_covid_ga',
    'search_additional_georgia_covid_y',
    'search_additional_CVS_vaccine',
    'search_additional_Coronavirus_disease_2019',
]

In [ ]:
column_cats = [cdh, cdh_age, pop, mob, misc, search, weather]
    
for category in column_cats:
    for column in category:
        prev_value = None
        for index, value in df[column].items():
            if pd.notna(value):
                prev_value = value
            else:
                if prev_value is not None:
                    df.at[index, column] = prev_value
                else:
                    df.at[index, column] = 0

## Setting Features

In [ ]:
features = cdh + cdh_age + pop + mob + misc + search + weather
if ADDITIONAL:
    features += search_additional

## If ADDITIONAL = TRUE
### we need to sum over the last 7 days from our original df
### we need to combine df and df_search

In [ ]:
if ADDITIONAL:
    numerical_columns = df.select_dtypes(include='number').columns
    non_numerical_columns = list(df.select_dtypes(exclude='number').columns) +[label]

    df_numerical = df[numerical_columns]
    df_non_numerical = df[non_numerical_columns]

    df_rolling_sum = df[cdh + cdh_age + pop + mob + misc + search + weather].rolling(window=7, min_periods=1).sum()

    df_rolling_sum = df_rolling_sum.reset_index( drop=True)

    df = pd.concat([df_non_numerical, df_rolling_sum], axis=1)
    
    df = pd.merge(df, df_search, on=date, how='inner')

## Determing Date Range 
### Vaccinations were not available for a while, so lets look at null ranges

In [ ]:
data_subset = df[[date,label]]

null_count_by_date = data_subset.groupby(date).apply(lambda x: x[label].isnull().sum())

plt.figure(figsize=(12, 6))
sns.lineplot(data=null_count_by_date)
plt.xlabel('Date')
plt.ylabel('Count of Null Values')
plt.title('Null Values in Vaccination Count by Date')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

longest_continuous_range_start = None
longest_continuous_range_end = None

current_continuous_range_start = None
current_continuous_range_end = None

for index, value in df[label].items():
    if pd.notna(value):
        if current_continuous_range_start is None:
            current_continuous_range_start = index
        current_continuous_range_end = index
    else:
        if (current_continuous_range_start is not None) and (current_continuous_range_end is not None):
            if (longest_continuous_range_start is None) or (current_continuous_range_end - current_continuous_range_start > longest_continuous_range_end - longest_continuous_range_start):
                longest_continuous_range_start = current_continuous_range_start
                longest_continuous_range_end = current_continuous_range_end
        current_continuous_range_start = None
        current_continuous_range_end = None

print(f"Start: Index {longest_continuous_range_start}")
print(f"End: Index {longest_continuous_range_end}")

start_date = df.at[longest_continuous_range_start, 'date']
end_date = df.at[longest_continuous_range_end, 'date']

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

In [ ]:
df = df.loc[longest_continuous_range_start:longest_continuous_range_end].reset_index(drop=True)

## Vaccination Uptake Delay

In [ ]:
date_column = 'date'
vaccine_column = label


plt.figure(figsize=(12, 6))
plt.plot(df[date_column], df[vaccine_column], label='Actual Vaccination Values', color='blue')
plt.xlabel('Date')
plt.ylabel('Vaccination Values')
plt.title('Actual Vaccination Values Over Time')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## When ADDITIONAL = FALSE
### Note the spikes occuring at 7, 14, 21, 28, incidicating some week like structure
## When ADDITIONAL = TRUE
### No obvious correlations

In [ ]:
acf_result = plot_acf(df[vaccine_column], lags=30)
plt.show()

## When ADDITIONAL = FALSE:
### No obvious date to use, so we can test with all of them
### AKA: weekly sum on monday, tuesday, wednesday ... and see which performs the best
### Will be done after dealing with features

In [ ]:
if not ADDITIONAL:
    
    df['day_of_week'] = df['date'].dt.day_name()

    day_counts = df.groupby('day_of_week')[label].sum()

    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = day_counts.reindex(days_of_week)

    plt.figure(figsize=(10, 6))
    day_counts.plot(kind='bar', color='skyblue')
    plt.title('Total Vaccinations by Day of the Week')
    plt.xlabel('Day of the Week')
    plt.ylabel('Total Number of Vaccinations')
    plt.show()

# EDA

In [ ]:
leads = []
for feature in features:
    cross_corr = np.correlate(df[feature].values, df[label].values, mode='full')
    time_lags = np.arange(-len(df[feature])+1, len(df[feature]))
    
        # Find the lag with maximum correlation
    max_corr_index = np.argmax(cross_corr)
    max_corr_lag = time_lags[max_corr_index]
    
    if max_corr_lag <= -1 and 'search_additional' in feature:

        # Plot cross-correlation
        plt.figure(figsize=(10, 5))
        plt.plot(time_lags, cross_corr, marker='o', linestyle='-')
        plt.title(f'Cross-Correlation between {feature} and Label')
        plt.xlabel('Time Lag')
        plt.ylabel('Cross-Correlation')
        plt.grid(True)
        plt.show()

    
    print(f"Optimal lag for {feature}: {max_corr_lag}")
    
    if max_corr_lag < 0:
        leads.append((feature, max_corr_lag))

In [ ]:
leads

### Storing old df without lag features and creating lagged df

In [ ]:
# df = df_og.copy()
df_og = df.copy()

In [ ]:
limit = -60
if ADDITIONAL:
    limit = -4
for feature, lag in leads:
    if lag >= limit:
        df[feature] = df[feature].shift(lag)
    
df = df.dropna()

In [ ]:
df.shape

In [ ]:
for group, columns in zip(["cdh", "cdh_age", "pop", "mob", "misc", "search", "search_additional","weather"],
                         [cdh, cdh_age, pop, mob, misc, search, search_additional, weather]):
    print(f"Group: {group}")
    
    summary = df[columns].describe()
    print("Summary Statistics:")
    print(summary)
    
    for column in columns:
        plt.figure(figsize=(10, 4))
        sns.histplot(df[column], kde=True)
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()
    
    correlation_matrix = df[columns].corr()
    print("Correlation Matrix:")
    print(correlation_matrix)
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title(f'Correlation Matrix: {group}')
    plt.show()
    

In [ ]:
groups = [cdh, cdh_age, pop, mob, misc, search, weather]
for group in groups:
    print(f"Group: {str(group)}")
    
    # Pearson Correlation
    pearson_correlations = {}
    for feature in group:
        if feature != label:
            correlation, _ = pearsonr(df[feature], df[label])
            pearson_correlations[feature] = correlation
    
    sorted_pearson_correlations = {k: v for k, v in sorted(pearson_correlations.items(), key=lambda item: abs(item[1]), reverse=True)}
    
    top_pearson_features = list(sorted_pearson_correlations.keys())[:10]
    top_pearson_values = list(sorted_pearson_correlations.values())[:10]
    
    plt.figure(figsize=(12, 4))
    plt.barh(top_pearson_features, top_pearson_values)
    plt.title(f'Top 10 Features by Pearson Correlation with {label}')
    plt.xlabel('Correlation')
    plt.show()
    
    # Spearman Correlation
    spearman_correlations = {}
    for feature in group:
        if feature != label:
            correlation, _ = spearmanr(df[feature], df[label])
            spearman_correlations[feature] = correlation
    
    sorted_spearman_correlations = {k: v for k, v in sorted(spearman_correlations.items(), key=lambda item: abs(item[1]), reverse=True)}
    
    top_spearman_features = list(sorted_spearman_correlations.keys())[:10]
    top_spearman_values = list(sorted_spearman_correlations.values())[:10]
    
    plt.figure(figsize=(12, 4))
    plt.barh(top_spearman_features, top_spearman_values)
    plt.title(f'Top 10 Features by Spearman Correlation with {label}')
    plt.xlabel('Correlation')
    plt.show()
    
    # Mutual Information
    mi_scores = mutual_info_regression(df[group], df[label], discrete_features='auto')
    
    sorted_mi_scores = {k: v for k, v in sorted(enumerate(mi_scores), key=lambda item: abs(item[1]), reverse=True)}
    
    top_mi_features = [group[k] for k in list(sorted_mi_scores.keys())[:10]]
    top_mi_scores = list(sorted_mi_scores.values())[:10]
    
    plt.figure(figsize=(12, 4))
    plt.barh(top_mi_features, top_mi_scores)
    plt.title(f'Top 10 Features by Mutual Information with {label}')
    plt.xlabel('Mutual Information Score')
    plt.show()

# Feature Selection

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = StandardScaler()

def get_top_k_features(df, k):
    top_k_features = []

    for train_index, test_index in tscv.split(df):
        train_data, test_data = df.iloc[train_index], df.iloc[test_index]
        X_train, X_test = train_data[features], test_data[features]
        y_train, y_test = train_data[label], test_data[label]

        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        rfe_selector = RFE(estimator=RandomForestRegressor(n_estimators=10), n_features_to_select=k)
        rfe_selector.fit(X_train_scaled, y_train)
        rfe_support = rfe_selector.support_

        skb_selector = SelectKBest(mutual_info_regression, k=k)
        skb_selector.fit(X_train_scaled, y_train)
        skb_support = skb_selector.get_support()

        lasso_selector = SelectFromModel(Lasso(alpha=0.01))
        lasso_selector.fit(X_train_scaled, y_train)
        lasso_support = lasso_selector.get_support()

        gb_selector = SelectFromModel(GradientBoostingRegressor(n_estimators=100))
        gb_selector.fit(X_train_scaled, y_train)
        gb_support = gb_selector.get_support()

        feature_support = {
            'RFE': rfe_support,
            'SelectKBest': skb_support,
            'LASSO': lasso_support,
            'GradientBoosting': gb_support,
        }

        votes = np.sum([feature_support[method] for method in feature_support.keys()], axis=0)
        top_k_features.extend(np.argsort(votes)[-k:])

    top_k_features = np.unique(top_k_features)[:k]

    return top_k_features

k_features_to_select = 50
top_k_features = get_top_k_features(df, k_features_to_select)

selected_feature_names = df.columns[top_k_features]
print(f"Top {k_features_to_select} Features: {selected_feature_names}")

### Can Test Mutiple feature selections below

In [ ]:
# features = list(selected_feature_names)
# # features = cdh + cdh_age + pop + mob + misc + search + weather
# features = search
# if ADDITIONAL:
#     features += search_additional

# Modeling without Rolling Sums per week
## ONLY RUNS IF ADDITIONAL = FALSE

In [ ]:
if not ADDITIONAL:
    
    tscv = TimeSeriesSplit(n_splits=10)

    for train_index, test_index in tscv.split(df):
        train_data, test_data = df.iloc[train_index], df.iloc[test_index]
        X_train, X_test = train_data[features], test_data[features]
        y_train, y_test = train_data[label], test_data[label]

        rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
        rf_regressor.fit(X_train, y_train)
        rf_predictions = rf_regressor.predict(X_test)

        mse_rf = mean_squared_error(y_test, rf_predictions)
        mae_rf = mean_absolute_error(y_test, rf_predictions)
        nmae_rf = normalized_mae(y_test, rf_predictions)
        nrmse_rf = normalized_rmse(y_test, rf_predictions)
        r2_rf = r2_score(y_test, rf_predictions)

        xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
        xgb_regressor.fit(X_train, y_train)
        xgb_predictions = xgb_regressor.predict(X_test)

        mse_xgb = mean_squared_error(y_test, xgb_predictions)
        mae_xgb = mean_absolute_error(y_test, xgb_predictions)
        nmae_xgb = normalized_mae(y_test, xgb_predictions)
        nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
        r2_xgb = r2_score(y_test, xgb_predictions)

        ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
        ada_regressor.fit(X_train, y_train)
        ada_predictions = ada_regressor.predict(X_test)

        mse_ada = mean_squared_error(y_test, ada_predictions)
        mae_ada = mean_absolute_error(y_test, ada_predictions)
        nmae_ada = normalized_mae(y_test, ada_predictions)
        nrmse_ada = normalized_rmse(y_test, ada_predictions)
        r2_ada = r2_score(y_test, ada_predictions)

        print("Random Forest Metrics:")
        print(f"Mean Squared Error: {mse_rf}")
        print(f"Mean Absolute Error: {mae_rf}")
        print(f"Normalized RMSE: {nrmse_rf}")
        print(f"Normalized MAE: {nmae_rf}")
        print(f"R-squared: {r2_rf}")

        print("\nXGBoost Metrics:")
        print(f"Mean Squared Error: {mse_xgb}")
        print(f"Mean Absolute Error: {mae_xgb}")
        print(f"Normalized RMSE: {nrmse_xgb}")
        print(f"Normalized MAE: {nmae_xgb}")
        print(f"R-squared: {r2_xgb}")

        print("\nAdaBoost Metrics:")
        print(f"Mean Squared Error: {mse_ada}")
        print(f"Mean Absolute Error: {mae_ada}")
        print(f"Normalized RMSE: {nrmse_ada}")
        print(f"Normalized MAE: {nmae_ada}")
        print(f"R-squared: {r2_ada}")

        # Plot the regression curves
        plt.figure(figsize=(10, 6))
        plt.plot(test_data['date'], y_test, label='Actual', color='blue')
        plt.plot(test_data['date'], rf_predictions, label='Random Forest', color='red')
        plt.plot(test_data['date'], xgb_predictions, label='XGBoost', color='green')
        plt.plot(test_data['date'], ada_predictions, label='AdaBoost', color='purple')
        plt.xlabel('Date')
        plt.ylabel('Label')
        plt.legend()
        plt.title('Regression Curves')
        plt.show()

# Modeling with Rolling Sums per week
## If ADDITIONAL = TRUE
### Rolling is already applied and only weekly values are used
## If ADDITIONAL = FALSE -> Two methods: 
### 1. rolling sum per 7 days
### 2. sum per week as index (weekly values)

In [ ]:
if ADDITIONAL:
    df_weekly_sum = df
else: 
    numerical_columns = df.select_dtypes(include='number').columns
    non_numerical_columns = df.select_dtypes(exclude='number').columns

    df_numerical = df[numerical_columns]
    df_non_numerical = df[non_numerical_columns]

    df_rolling_sum = df[features + [label] + ['day_of_week']].groupby('day_of_week').rolling(window=7, min_periods=1).sum()

    df_rolling_sum = df_rolling_sum.reset_index( drop=True)

    df_weekly_sum = pd.concat([df_non_numerical, df_rolling_sum], axis=1)

plt.figure(figsize=(10, 6))
plt.plot(df_weekly_sum.index, df_weekly_sum[label], marker='o', linestyle='-', color='b')
plt.title('Weekly Sum of Vaccinations')
plt.xlabel('Date')
plt.ylabel('Weekly Sum of Vaccinations')
plt.grid(True)
plt.show()


## Method 1: 
### If ADDITIONAL = TRUE: Modeling with rolling sum values per week (every week is a datapoint)
### Else: Modeling with all rolling sum values (every day is a datapoint)

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

df_weekly_sum.reset_index(drop=True, inplace=True)

cumulative_metrics_rf = {'MSE': 0, 'MAE': 0, 'NMAE': 0, 'NRMSE': 0, 'R2': 0}
cumulative_metrics_xgb = {'MSE': 0, 'MAE': 0, 'NMAE': 0, 'NRMSE': 0, 'R2': 0}
cumulative_metrics_ada = {'MSE': 0, 'MAE': 0, 'NMAE': 0, 'NRMSE': 0, 'R2': 0}

cumulative_predictions_rf = []
cumulative_predictions_xgb = []
cumulative_predictions_ada = []

for train_index, test_index in tscv.split(df_weekly_sum):
    print(train_index, test_index)
    train_data, test_data = df_weekly_sum.iloc[train_index], df_weekly_sum.iloc[test_index]
    X_train, X_test = train_data[features], test_data[features]
    y_train, y_test = train_data[label], test_data[label]

    # Random Forest
    rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_regressor.fit(X_train, y_train)
    rf_predictions = rf_regressor.predict(X_test)

    # XGBoost
    xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
    xgb_regressor.fit(X_train, y_train)
    xgb_predictions = xgb_regressor.predict(X_test)

    # AdaBoost
    ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
    ada_regressor.fit(X_train, y_train)
    ada_predictions = ada_regressor.predict(X_test)

    # Append predictions to cumulative arrays
    cumulative_predictions_rf.extend(rf_predictions)
    cumulative_predictions_xgb.extend(xgb_predictions)
    cumulative_predictions_ada.extend(ada_predictions)

    # Calculate metrics for Random Forest
    mse_rf = mean_squared_error(y_test, rf_predictions)
    mae_rf = mean_absolute_error(y_test, rf_predictions)
    nmae_rf = normalized_mae(y_test, rf_predictions)
    nrmse_rf = normalized_rmse(y_test, rf_predictions)
    r2_rf = r2_score(y_test, rf_predictions)

    # Update cumulative metrics for Random Forest
    cumulative_metrics_rf['MSE'] += mse_rf
    cumulative_metrics_rf['MAE'] += mae_rf
    cumulative_metrics_rf['NMAE'] += nmae_rf
    cumulative_metrics_rf['NRMSE'] += nrmse_rf
    cumulative_metrics_rf['R2'] += r2_rf

    # Calculate metrics for XGBoost
    mse_xgb = mean_squared_error(y_test, xgb_predictions)
    mae_xgb = mean_absolute_error(y_test, xgb_predictions)
    nmae_xgb = normalized_mae(y_test, xgb_predictions)
    nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
    r2_xgb = r2_score(y_test, xgb_predictions)

    # Update cumulative metrics for XGBoost
    cumulative_metrics_xgb['MSE'] += mse_xgb
    cumulative_metrics_xgb['MAE'] += mae_xgb
    cumulative_metrics_xgb['NMAE'] += nmae_xgb
    cumulative_metrics_xgb['NRMSE'] += nrmse_xgb
    cumulative_metrics_xgb['R2'] += r2_xgb

    # Calculate metrics for AdaBoost
    mse_ada = mean_squared_error(y_test, ada_predictions)
    mae_ada = mean_absolute_error(y_test, ada_predictions)
    nmae_ada = normalized_mae(y_test, ada_predictions)
    nrmse_ada = normalized_rmse(y_test, ada_predictions)
    r2_ada = r2_score(y_test, ada_predictions)

    # Update cumulative metrics for AdaBoost
    cumulative_metrics_ada['MSE'] += mse_ada
    cumulative_metrics_ada['MAE'] += mae_ada
    cumulative_metrics_ada['NMAE'] += nmae_ada
    cumulative_metrics_ada['NRMSE'] += nrmse_ada
    cumulative_metrics_ada['R2'] += r2_ada

    # Plot regression curves for each time split
    plt.figure(figsize=(10, 6))
    plt.plot(test_data.index, y_test, label='Actual', color='blue')
    plt.plot(test_data.index, rf_predictions, label='Random Forest', color='red')
    plt.plot(test_data.index, xgb_predictions, label='XGBoost', color='green')
    plt.plot(test_data.index, ada_predictions, label='AdaBoost', color='purple')
    plt.xlabel('Index')
    plt.ylabel('Label')
    plt.legend()
    plt.title(f'Regression Curves - Time Split {train_index.max() + 1} to {test_index.max()}')
    plt.show()

# Create combined plot for overall forecast
plt.figure(figsize=(15, 8))
plt.plot(df_weekly_sum.iloc[15:,:].index, df_weekly_sum.iloc[15:,:][label], label='Actual', color='blue')
plt.plot(df_weekly_sum.iloc[15:,:].index, cumulative_predictions_rf, label='Random Forest', color='red')
plt.plot(df_weekly_sum.iloc[15:,:].index, cumulative_predictions_xgb, label='XGBoost', color='green')
plt.plot(df_weekly_sum.iloc[15:,:].index, cumulative_predictions_ada, label='AdaBoost', color='purple')
plt.xlabel('Index')
plt.ylabel('Label')
plt.legend()
plt.title('Combined Regression Curves - Overall Forecast')
plt.show()

# Calculate average metrics over all time splits
num_splits = tscv.get_n_splits(df_weekly_sum)
average_metrics_rf = {key: value / num_splits for key, value in cumulative_metrics_rf.items()}
average_metrics_xgb = {key: value / num_splits for key, value in cumulative_metrics_xgb.items()}
average_metrics_ada = {key: value / num_splits for key, value in cumulative_metrics_ada.items()}

# Print average metrics
print("\nAverage Metrics - Random Forest:")
print(average_metrics_rf)

print("\nAverage Metrics - XGBoost:")
print(average_metrics_xgb)

print("\nAverage Metrics - AdaBoost:")
print(average_metrics_ada)


### Code below is given if expansion training is preferred over time series split

In [ ]:
# label = 'new_vaccine_doses_administered'
# tscv = TimeSeriesSplit(n_splits=5)

# df_weekly_sum.reset_index(drop=True, inplace=True)

# num_expansions = 10
# expansion_size = len(df_weekly_sum) // num_expansions

# for expansion in range(num_expansions):
#     train_data = df_weekly_sum.iloc[:expansion_size * (expansion + 1)]
#     test_data = df_weekly_sum.iloc[expansion_size * (expansion + 1):expansion_size * (expansion + 2)]

#     X_train, X_test = train_data[features], test_data[features]
#     y_train, y_test = train_data[label], test_data[label]

#     rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
#     rf_regressor.fit(X_train, y_train)
#     rf_predictions = rf_regressor.predict(X_test)

#     xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
#     xgb_regressor.fit(X_train, y_train)
#     xgb_predictions = xgb_regressor.predict(X_test)

#     ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
#     ada_regressor.fit(X_train, y_train)
#     ada_predictions = ada_regressor.predict(X_test)

#     mse_rf = mean_squared_error(y_test, rf_predictions)
#     mae_rf = mean_absolute_error(y_test, rf_predictions)
#     nmae_rf = normalized_mae(y_test, rf_predictions)
#     nrmse_rf = normalized_rmse(y_test, rf_predictions)
#     r2_rf = r2_score(y_test, rf_predictions)

#     mse_xgb = mean_squared_error(y_test, xgb_predictions)
#     mae_xgb = mean_absolute_error(y_test, xgb_predictions)
#     nmae_xgb = normalized_mae(y_test, xgb_predictions)
#     nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
#     r2_xgb = r2_score(y_test, xgb_predictions)

#     mse_ada = mean_squared_error(y_test, ada_predictions)
#     mae_ada = mean_absolute_error(y_test, ada_predictions)
#     nmae_ada = normalized_mae(y_test, ada_predictions)
#     nrmse_ada = normalized_rmse(y_test, ada_predictions)
#     r2_ada = r2_score(y_test, ada_predictions)

#     print(f"\nExpansion {expansion + 1} Metrics:")
#     print("\nRandom Forest Metrics:")
#     print(f"Mean Squared Error: {mse_rf}")
#     print(f"Mean Absolute Error: {mae_rf}")
#     print(f"Normalized RMSE: {nrmse_rf}")
#     print(f"Normalized MAE: {nmae_rf}")
#     print(f"R-squared: {r2_rf}")

#     print("\nXGBoost Metrics:")
#     print(f"Mean Squared Error: {mse_xgb}")
#     print(f"Mean Absolute Error: {mae_xgb}")
#     print(f"Normalized RMSE: {nrmse_xgb}")
#     print(f"Normalized MAE: {nmae_xgb}")
#     print(f"R-squared: {r2_xgb}")

#     print("\nAdaBoost Metrics:")
#     print(f"Mean Squared Error: {mse_ada}")
#     print(f"Mean Absolute Error: {mae_ada}")
#     print(f"Normalized RMSE: {nrmse_ada}")
#     print(f"Normalized MAE: {nmae_ada}")
#     print(f"R-squared: {r2_ada}")

#     plt.figure(figsize=(10, 6))
#     plt.plot(test_data.index, y_test, label='Actual', color='blue')
#     plt.plot(test_data.index, rf_predictions, label='Random Forest', color='red')
#     plt.plot(test_data.index, xgb_predictions, label='XGBoost', color='green')
#     plt.plot(test_data.index, ada_predictions, label='AdaBoost', color='purple')
#     plt.xlabel('Index')
#     plt.ylabel('Label')
#     plt.legend()
#     plt.title(f'Regression Curves - Expansion {expansion + 1}')
#     plt.show()


## Method 2: Using Rolling Sum for weekly data points
### Will only run if Additional = False

In [ ]:
if not ADDITIONAL:
    
    plt.figure(figsize=(10, 6))

    for day in df_weekly_sum['day_of_week'].unique():
        df_day = df[df['day_of_week'] == day]
        plt.plot(df_day.index, df_day[label], marker='o', linestyle='-', label=day)

    plt.title('Daily Vaccinations for Each Day of the Week')
    plt.xlabel('Date')
    plt.ylabel('Daily Vaccinations')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
if  not ADDITIONAL:

    tscv = TimeSeriesSplit(n_splits=5)

    results_df = pd.DataFrame(columns=['Day', 'Model', 'MSE', 'MAE', 'NMAE', 'NRMSE', 'R2'])

    for day in df_weekly_sum['day_of_week'].unique():

        df_day = df_weekly_sum[df_weekly_sum['day_of_week'] == day]

        for train_index, test_index in tscv.split(df_day):
            train_data, test_data = df_day.iloc[train_index], df_day.iloc[test_index]
            X_train, X_test = train_data[features], test_data[features]
            y_train, y_test = train_data[label], test_data[label]

            rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
            rf_regressor.fit(X_train, y_train)
            rf_predictions = rf_regressor.predict(X_test)

            xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
            xgb_regressor.fit(X_train, y_train)
            xgb_predictions = xgb_regressor.predict(X_test)

            ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
            ada_regressor.fit(X_train, y_train)
            ada_predictions = ada_regressor.predict(X_test)

            mse_rf = mean_squared_error(y_test, rf_predictions)
            mae_rf = mean_absolute_error(y_test, rf_predictions)
            nmae_rf = normalized_mae(y_test, rf_predictions)
            nrmse_rf = normalized_rmse(y_test, rf_predictions)
            r2_rf = r2_score(y_test, rf_predictions)

            mse_xgb = mean_squared_error(y_test, xgb_predictions)
            mae_xgb = mean_absolute_error(y_test, xgb_predictions)
            nmae_xgb = normalized_mae(y_test, xgb_predictions)
            nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
            r2_xgb = r2_score(y_test, xgb_predictions)

            mse_ada = mean_squared_error(y_test, ada_predictions)
            mae_ada = mean_absolute_error(y_test, ada_predictions)
            nmae_ada = normalized_mae(y_test, ada_predictions)
            nrmse_ada = normalized_rmse(y_test, ada_predictions)
            r2_ada = r2_score(y_test, ada_predictions)

            results_df = pd.concat([results_df, pd.DataFrame({
                'Day': [day],
                'Model': ['Random Forest'],
                'MSE': [mse_rf],
                'MAE': [mae_rf],
                'NMAE': [nmae_rf],
                'NRMSE': [nrmse_rf],
                'R2': [r2_rf]
            })], ignore_index=True)

            results_df = pd.concat([results_df, pd.DataFrame({
                'Day': [day],
                'Model': ['XGBoost'],
                'MSE': [mse_xgb],
                'MAE': [mae_xgb],
                'NMAE': [nmae_xgb],
                'NRMSE': [nrmse_xgb],
                'R2': [r2_xgb]
            })], ignore_index=True)

            results_df = pd.concat([results_df, pd.DataFrame({
                'Day': [day],
                'Model': ['AdaBoost'],
                'MSE': [mse_ada],
                'MAE': [mae_ada],
                'NMAE': [nmae_ada],
                'NRMSE': [nrmse_ada],
                'R2': [r2_ada]
            })], ignore_index=True)

            # Plot the regression curves
            plt.figure(figsize=(10, 6))
            plt.plot(test_data['date'], y_test, label='Actual', color='blue')
            plt.plot(test_data['date'], rf_predictions, label='Random Forest', color='red')
            plt.plot(test_data['date'], xgb_predictions, label='XGBoost', color='green')
            plt.plot(test_data['date'], ada_predictions, label='AdaBoost', color='purple')
            plt.xlabel('Date')
            plt.ylabel('Label')
            plt.legend()
            plt.title(f'Regression Curves for Day {day}')
            plt.show()

In [ ]:
if not ADDITIONAL:
    results_df

In [ ]:
if not ADDITIONAL:
    
    for model in results_df['Model'].unique():
        model_results = results_df[results_df['Model'] == model]
        plt.figure(figsize=(10, 6))
        for day in df_weekly_sum['day_of_week'].unique():
            day_results = model_results[model_results['Day'] == day]
            plt.plot(day_results['Day'], day_results['NMAE'], label=f'{model} - {day}')

        plt.title(f'Normalized MAE for {model} by Day of Week')
        plt.xlabel('Day of Week')
        plt.ylabel('Normalized MAE')
        plt.legend()
        plt.show()

# Genetic Programming

## Select Label

In [ ]:
label = 'new_vaccine_doses_administered'

## Initialize Toolbox
### Primitive, Terminals, Fitness, Mutation, Mating, Population Generation, and Selection Functions

In [ ]:
def exp_func(x):
    return np.exp(np.clip(x, -700, 700))

pset = gp.PrimitiveSet("MAIN", arity=len(features))
pset.addPrimitive(np.add, arity=2)
pset.addPrimitive(np.subtract, arity=2)
pset.addPrimitive(np.multiply, arity=2)
pset.addPrimitive(np.negative, arity=1)
pset.addPrimitive(np.sin, arity=1)
pset.addPrimitive(np.cos, arity=1)
# pset.addTerminal(1.0, name="constant_one")
# pset.addTerminal(2.0, name="constant_two")
# pset.addTerminal(3.0, name="constant_three")
# pset.addPrimitive(exp_func, arity=1, name="exp")


def evalSymbReg(individual, X_train, y_train):
    func = gp.compile(expr=individual, pset=pset)
    predictions = func(*X_train.T)
    
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_train, predictions)
    r2 = r2_score(y_train, predictions)
    
    return rmse, mae, r2

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, 1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=25)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutNodeReplacement, pset=pset)

## Set Parameters, Run Generations, and View Results
### Current Values are 50 Gen and 1000 Population Size
### DECREASE to run faster

In [ ]:
gen = range(50)
avg_list = []
max_list = []
min_list = []
print('start population')

pop = toolbox.population(n=1000)
print('Population created')

for g in tqdm(gen, desc="Generations"):
    if g%25 == 0:
        print('Generation ' + str(g))
    

    all_metrics = np.zeros((len(pop), 3))
    
    for i, ind in enumerate(pop):
        X_train, y_train = df[features].values, df[label].values
        
        metrics = np.array(toolbox.evaluate(ind, X_train, y_train))
        all_metrics[i, :] = metrics
#     print('Evaluation Done')
    

    avg_metrics = np.mean(all_metrics, axis=0)
    
    for ind in pop:
        ind.fitness.values = tuple(avg_metrics)


    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.3:
            toolbox.mutate(mutant)
            del mutant.fitness.values



    for mutant in offspring:
        X_mutant, y_mutant = df[features].values, df[label].values
        mutant.fitness.values = toolbox.evaluate(mutant, X_mutant, y_mutant)
    

    pop[:] = offspring


    fits = [ind.fitness.values for ind in pop]

    length = len(pop)
    sum2 = sum(np.array(fits)**2)
    std = np.sqrt(abs(sum2 / length - np.array(avg_metrics)**2))
    g_max = max(fits, key=lambda x: x[2])  # Maximize R-squared
    g_min = min(fits, key=lambda x: x[0])  # Minimize RMSE

    max_list.append(g_max)
    min_list.append(g_min)
    avg_list.append(avg_metrics)



best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


func = gp.compile(expr=best_ind, pset=pset)
print(func)
predictions = func(*X_train.T)

plt.figure()
plt.plot(df.index, predictions, label='Predicted')
plt.plot(df.index, y_train, label='Actual')
plt.xlabel('Date')
plt.ylabel('Label')
plt.legend()
plt.title('Best Individual Prediction vs Actual')
plt.show()


plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.plot(gen, [item[0] for item in min_list], label="minimum RMSE")
plt.plot(gen, [item[0] for item in max_list], label="maximum RMSE")
plt.xlabel("Generation")
plt.ylabel("RMSE")
plt.legend(loc="upper right")
plt.title("Evolution of RMSE")

plt.subplot(3, 1, 2)
plt.plot(gen, [item[1] for item in min_list], label="minimum MAE")
plt.plot(gen, [item[1] for item in max_list], label="maximum MAE")
plt.xlabel("Generation")
plt.ylabel("MAE")
plt.legend(loc="upper right")
plt.title("Evolution of MAE")

plt.subplot(3, 1, 3)
plt.plot(gen, [item[2] for item in min_list], label="minimum R-squared")
plt.plot(gen, [item[2] for item in max_list], label="maximum R-squared")
plt.xlabel("Generation")
plt.ylabel("R-squared")
plt.legend(loc="upper right")
plt.title("Evolution of R-squared")

plt.tight_layout()
plt.show()


pareto_front = tools.sortNondominated(pop, len(pop), first_front_only=True)[0]

sorted_pareto_front = sorted(pareto_front, key=lambda ind: ind.fitness.values[0])

top_20_individuals = sorted_pareto_front[:20]

for i, ind in enumerate(top_20_individuals):
    print(f"Top {i + 1} Individual: {ind}, Fitness: {ind.fitness.values}")

plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.plot(gen, [item[0] for item in min_list], label="minimum RMSE")
plt.plot(gen, [item[0] for item in max_list], label="maximum RMSE")
plt.xlabel("Generation")
plt.ylabel("RMSE")
plt.legend(loc="upper right")
plt.title("Evolution of RMSE")

plt.subplot(3, 1, 2)
plt.plot(gen, [item[1] for item in min_list], label="minimum MAE")
plt.plot(gen, [item[1] for item in max_list], label="maximum MAE")
plt.xlabel("Generation")
plt.ylabel("MAE")
plt.legend(loc="upper right")
plt.title("Evolution of MAE")

plt.subplot(3, 1, 3)
plt.plot(gen, [item[2] for item in min_list], label="minimum R-squared")
plt.plot(gen, [item[2] for item in max_list], label="maximum R-squared")
plt.xlabel("Generation")
plt.ylabel("R-squared")
plt.legend(loc="upper right")
plt.title("Evolution of R-squared")

plt.tight_layout()
plt.show()


# Predicting Peaks and Falls
## Utilizing our Models' Strengths

### Plot Vaccinations

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.index, df[label], marker='o', linestyle='-', color='b')
plt.title('Weekly Sum of Vaccinations')
plt.xlabel('Date')
plt.ylabel('Weekly Sum of Vaccinations')
plt.grid(True)
plt.show()


## Determine the best performance ranges for our model 
### Best_idx is populated with train and test indices for further analysis
### Indices are chosen based on r^2 value > 0.5

In [ ]:
indexes = []
for test_size in range(3,7):
    for train_end in range(10,55):
        indexes.append(([i for i in range(0,train_end)],[i for i in range(train_end,train_end+test_size)]))
    
best_idx = []

for train_index, test_index in tqdm(indexes):

    train_data, test_data = df.iloc[train_index], df.iloc[test_index]
    X_train, X_test = train_data[features], test_data[features]
    y_train, y_test = train_data[label], test_data[label]

    rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_regressor.fit(X_train, y_train)
    rf_predictions = rf_regressor.predict(X_test)

    xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
    xgb_regressor.fit(X_train, y_train)
    xgb_predictions = xgb_regressor.predict(X_test)
    
    ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
    ada_regressor.fit(X_train, y_train)
    ada_predictions = ada_regressor.predict(X_test)
    
    mse_rf = mean_squared_error(y_test, rf_predictions)
    mae_rf = mean_absolute_error(y_test, rf_predictions)
    nmae_rf = normalized_mae(y_test, rf_predictions)
    nrmse_rf = normalized_rmse(y_test, rf_predictions)
    r2_rf = r2_score(y_test, rf_predictions)

    mse_xgb = mean_squared_error(y_test, xgb_predictions)
    mae_xgb = mean_absolute_error(y_test, xgb_predictions)
    nmae_xgb = normalized_mae(y_test, xgb_predictions)
    nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
    r2_xgb = r2_score(y_test, xgb_predictions)

    mse_ada = mean_squared_error(y_test, ada_predictions)
    mae_ada = mean_absolute_error(y_test, ada_predictions)
    nmae_ada = normalized_mae(y_test, ada_predictions)
    nrmse_ada = normalized_rmse(y_test, ada_predictions)
    r2_ada = r2_score(y_test, ada_predictions)
    
    if r2_rf >= 0.5 or r2_xgb >= 0.5 or r2_ada >= 0.5:
        best_idx.append((train_index, test_index))



### Store results from our best index ranges

In [ ]:

results_list = []

for train_index, test_index in best_idx:

    print(train_index, test_index)
    train_data, test_data = df.iloc[train_index], df.iloc[test_index]
    X_train, X_test = train_data[features], test_data[features]
    y_train, y_test = train_data[label], test_data[label]

    # Random Forest
    rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_regressor.fit(X_train, y_train)
    rf_predictions = rf_regressor.predict(X_test)

    # XGBoost
    xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
    xgb_regressor.fit(X_train, y_train)
    xgb_predictions = xgb_regressor.predict(X_test)

    # AdaBoost
    ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
    ada_regressor.fit(X_train, y_train)
    ada_predictions = ada_regressor.predict(X_test)

    r2_rf = r2_score(y_test, rf_predictions)
    r2_xgb = r2_score(y_test, xgb_predictions)
    r2_ada = r2_score(y_test, ada_predictions)

    results_list.append((train_index, test_index, r2_rf, 'Random Forest'))
    results_list.append((train_index, test_index, r2_xgb, 'XGBoost'))
    results_list.append((train_index, test_index, r2_ada, 'AdaBoost'))

results_list.sort(key=lambda x: x[2], reverse=True)

### Print Results and Best models for those ranges

In [ ]:

for result in results_list:
    if result[2] >= 0.5:
        print(f"Train Index: {result[0]}, Test Index: {result[1]}, R-squared: {result[2]}, Model Type: {result[3]}")
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')


## Output Curves for Best Models on Best Ranges

In [ ]:
# Filter through results ranges for presentation
final_indexes = results_list[:8]

for train_index, test_index, _, model_type in final_indexes:

    print(train_index, test_index)
    train_data, test_data = df.iloc[train_index], df.iloc[test_index]
    X_train, X_test = train_data[features], test_data[features]
    y_train, y_test = train_data[label], test_data[label]
    
    if model_type == 'Random Forest':

        rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
        rf_regressor.fit(X_train, y_train)
        rf_predictions = rf_regressor.predict(X_test)

        mse_rf = mean_squared_error(y_test, rf_predictions)
        mae_rf = mean_absolute_error(y_test, rf_predictions)
        nmae_rf = normalized_mae(y_test, rf_predictions)
        nrmse_rf = normalized_rmse(y_test, rf_predictions)
        r2_rf = r2_score(y_test, rf_predictions)
    
        print("\nRandom Forest Metrics:")
        print(f"Mean Squared Error: {mse_rf}")
        print(f"Mean Absolute Error: {mae_rf}")
        print(f"Normalized RMSE: {nrmse_rf}")
        print(f"Normalized MAE: {nmae_rf}")
        print(f"R-squared: {r2_rf}")
    
    if model_type == 'XGBoost':
    
        xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
        xgb_regressor.fit(X_train, y_train)
        xgb_predictions = xgb_regressor.predict(X_test)

        mse_xgb = mean_squared_error(y_test, xgb_predictions)
        mae_xgb = mean_absolute_error(y_test, xgb_predictions)
        nmae_xgb = normalized_mae(y_test, xgb_predictions)
        nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
        r2_xgb = r2_score(y_test, xgb_predictions)

        print("\nXGBoost Metrics:")
        print(f"Mean Squared Error: {mse_xgb}")
        print(f"Mean Absolute Error: {mae_xgb}")
        print(f"Normalized RMSE: {nrmse_xgb}")
        print(f"Normalized MAE: {nmae_xgb}")
        print(f"R-squared: {r2_xgb}")
    
    if model_type == 'AdaBoost':
        ada_regressor = AdaBoostRegressor(n_estimators=500, random_state=42)
        ada_regressor.fit(X_train, y_train)
        ada_predictions = ada_regressor.predict(X_test)

        mse_ada = mean_squared_error(y_test, ada_predictions)
        mae_ada = mean_absolute_error(y_test, ada_predictions)
        nmae_ada = normalized_mae(y_test, ada_predictions)
        nrmse_ada = normalized_rmse(y_test, ada_predictions)
        r2_ada = r2_score(y_test, ada_predictions)

        print("\nAdaBoost Metrics:")
        print(f"Mean Squared Error: {mse_ada}")
        print(f"Mean Absolute Error: {mae_ada}")
        print(f"Normalized RMSE: {nrmse_ada}")
        print(f"Normalized MAE: {nmae_ada}")
        print(f"R-squared: {r2_ada}")

    plt.figure(figsize=(10, 6))
    plt.plot(test_data.index, y_test, label='Actual', color='blue')
    if model_type == 'Random Forest':
        plt.plot(test_data.index, rf_predictions, label='Random Forest', color='red')
    if model_type == 'XGBoost':
        plt.plot(test_data.index, xgb_predictions, label='XGBoost', color='green')
    if model_type == 'AdaBoost':
        plt.plot(test_data.index, ada_predictions, label='AdaBoost', color='purple')
    plt.xlabel('Index')
    plt.ylabel('Label')
    plt.legend()
    plt.title(f'Regression Curves')
    plt.show()

# Interpretability with SHAP
## Calculates shapley values for individual models on their best ranges and outputs force plots for each week

In [ ]:
shap_values_dict = {}

i = 0

for train_index, test_index, _, model_type in final_indexes:
    print(train_index, test_index)

    train_data, test_data = df.iloc[train_index], df.iloc[test_index]
    X_train, X_test = train_data[features], test_data[features]
    y_train, y_test = train_data[label], test_data[label]

    if model_type == 'Random Forest':
        rf_regressor = RandomForestRegressor(n_estimators=500, random_state=42)
        rf_regressor.fit(X_train, y_train)
        rf_predictions = rf_regressor.predict(X_test)

        mse_rf = mean_squared_error(y_test, rf_predictions)
        mae_rf = mean_absolute_error(y_test, rf_predictions)
        nmae_rf = normalized_mae(y_test, rf_predictions)
        nrmse_rf = normalized_rmse(y_test, rf_predictions)
        r2_rf = r2_score(y_test, rf_predictions)

        print("\nRandom Forest Metrics:")
        print(f"Mean Squared Error: {mse_rf}")
        print(f"Mean Absolute Error: {mae_rf}")
        print(f"Normalized RMSE: {nrmse_rf}")
        print(f"Normalized MAE: {nmae_rf}")
        print(f"R-squared: {r2_rf}")

        explainer_rf = shap.TreeExplainer(rf_regressor)
        shap_values_rf = explainer_rf.shap_values(X_test)

        shap_values_dict[i] = shap_values_rf

        for j in range(len(test_index)):
            force_plot = shap.force_plot(explainer_rf.expected_value, shap_values_rf[j, :], X_test.iloc[j, :], feature_names=features)
            display(HTML(force_plot.html()))
            
    if model_type == 'XGBoost':
        xgb_regressor = XGBRegressor(n_estimators=500, random_state=42)
        xgb_regressor.fit(X_train, y_train)
        xgb_predictions = xgb_regressor.predict(X_test)

        mse_xgb = mean_squared_error(y_test, xgb_predictions)
        mae_xgb = mean_absolute_error(y_test, xgb_predictions)
        nmae_xgb = normalized_mae(y_test, xgb_predictions)
        nrmse_xgb = normalized_rmse(y_test, xgb_predictions)
        r2_xgb = r2_score(y_test, xgb_predictions)

        print("\nXGBoost Metrics:")
        print(f"Mean Squared Error: {mse_xgb}")
        print(f"Mean Absolute Error: {mae_xgb}")
        print(f"Normalized RMSE: {nrmse_xgb}")
        print(f"Normalized MAE: {nmae_xgb}")
        print(f"R-squared: {r2_xgb}")

        explainer_xgb = shap.Explainer(xgb_regressor)
        shap_values_xgb = explainer_xgb.shap_values(X_test)

        shap_values_dict[i] = shap_values_xgb

        for j in range(len(test_index)):
            force_plot = shap.force_plot(explainer_xgb.expected_value, shap_values_xgb[j, :], X_test.iloc[j, :], feature_names=features)
            display(HTML(force_plot.html()))
    i += 1
